In [ ]:
### modules　related to system
import time
import logging
import warnings
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')
import pandas as pd, numpy as np
import math, json, gc, random, os, sys
import matplotlib.pyplot as plt
from google.colab import files


### modules　related to models
from torch.cuda.amp import autocast, GradScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as dataset
from sklearn.model_selection import train_test_split
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


from logging import Logger, getLogger, INFO, FileHandler,  Formatter,  StreamHandler

def init_logger(log_file='logfile.log'):
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter('%(asctime)s: %(message)s   :::%(name)s:%(lineno)s %(funcName)s [%(levelname)s] '))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    logger.propagate = False
    return logger


LOGGER = init_logger()

In [ ]:
df = pd.read_csv('/content/kaaaa.csv', )
df.density = df.density.apply(eval)
df.weight = df.weight.apply(eval)

train_df, val_df = train_test_split(df, train_size=0.8, shuffle= True)

In [ ]:
input = 99
layer1 = 700
layer2 = 700
interval = 49

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(input, layer1)
        self.fc2 = nn.Linear(layer1, layer2)
        self.fc3 = nn.Linear(layer2, interval)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.dropout(x)
        x = F.relu(self.fc2(x))
        x = F.dropout(x)
        x = F.relu(self.fc3(x))
        return x

In [ ]:
class Dataset(dataset.Dataset):
    def __init__(self,df):        
        self.df=df
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        weight =  torch.tensor(df.weight.iloc[idx])
        density = torch.tensor(df.density.iloc[idx])
        return  density, weight

In [ ]:
MAX_EPOCH = 1000

loaders = {
    "train": dataset.DataLoader(Dataset(train_df),
                              batch_size=512, 
                              shuffle=True, 
                              num_workers=2, 
                              pin_memory=True, 
                              drop_last=True),
    "valid": dataset.DataLoader(Dataset(val_df),
                              batch_size=512, 
                              shuffle=False,
                              num_workers=2,
                              pin_memory=True,
                              drop_last=False),
}

# model
model = Model().to(device)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.00006)

# Scheduler
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=MAX_EPOCH)

# Loss
criterion = nn.MSELoss()

scaler = GradScaler()

best_loss=1000000


In [ ]:
FREQ = 50

for epoch in range(MAX_EPOCH):
        sum_loss =0
        model.train()
        gc.collect()
        for x, y in loaders['train']:
            x = x.to(device)
            y = y.to(device)
            output = model(x)
            loss = criterion(output, y)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            sum_loss +=loss.item()
        scheduler.step()
        train_loss=sum_loss/len(loaders['train'])
        sum_loss=0
        model.eval()
        gc.collect()
        for inputs, labels in loaders['valid']:
            labels = labels.to(device)
            inputs = inputs.to(device)
            with torch.no_grad():
                output = model(x) 
                loss = criterion(output, y)
                sum_loss+=loss.item()
        valid_loss=sum_loss/len(loaders['valid'])
        if epoch % FREQ == 0:
          LOGGER.info(f'current epoch:{epoch}')
          LOGGER.info(f'loss:{train_loss}')
          LOGGER.info(f'valid_loss:{valid_loss}')
        if valid_loss < best_loss:
            best_loss=valid_loss
            best_epoch=epoch
            torch.save(model.to('cpu').state_dict(), 'best.pth')

LOGGER.info('========= training result ==========')
LOGGER.info(f'best epoch:{best_epoch}')
LOGGER.info(f'best loss:{best_loss}')

files.download('best.pth')

current epoch:0
loss:0.001060599577613175
valid_loss:0.000907083653146401
current epoch:50
loss:4.703193813579024e-05
valid_loss:4.525846634351183e-05
current epoch:100
loss:2.5514404894368325e-05
valid_loss:2.4557261895097326e-05
current epoch:150
loss:1.8837226889445446e-05
valid_loss:1.8692654521146324e-05
current epoch:200
loss:1.5092612427127148e-05
valid_loss:1.5665396858821623e-05
current epoch:250
loss:1.278668423765339e-05
valid_loss:1.2522506040113512e-05
current epoch:300
loss:1.1336911094466423e-05
valid_loss:1.1100270967290271e-05
current epoch:350
loss:9.94702687811306e-06
valid_loss:1.0127208952326328e-05
current epoch:400
loss:8.826652057385737e-06
valid_loss:8.872817943483824e-06
current epoch:450
loss:8.028775222815707e-06
valid_loss:8.092146799754119e-06
current epoch:500
loss:7.249231332804941e-06
valid_loss:7.167565399868181e-06
current epoch:550
loss:6.7504850319112715e-06
valid_loss:6.758308700227644e-06
current epoch:600
loss:6.422130159730191e-06
valid_loss:6.2

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model = Model().to(device)
checkpoint = torch.load('/content/best.pth')
model.load_state_dict(checkpoint)
model.eval()
submission = pd.DataFrame(index=[], columns=['expected_weight', 'true'])
for i, (x,y)   in tqdm(enumerate(loaders['valid']), total=len(loaders['valid'])):
    with torch.no_grad():
        prediction = model(x).to('cpu').numpy().copy().tolist()
        y = y.to('cpu').numpy().copy().tolist()
        for i in range(len(prediction)):
          submission = submission.append({'expected_weight': prediction[i], 'true' : y[i]}, ignore_index=True)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
submission.to_csv('submission.csv')
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>